In [1]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [2]:
# Lineitem

l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX",
                "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT", "L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()

l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../data/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

In [3]:
start_time = time.time()

df_filter_1 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_filter_1['before_1'] = (df_filter_1.l_quantity)
df_filter_1['before_2'] = ((df_filter_1.l_discount) * 0.5)
df_filter_1['before_3'] = (df_filter_1.l_extendedprice)
df_filter_1['before_4'] = (df_filter_1.l_tax)
df_filter_1['before_5'] = (df_filter_1.l_quantity)
df_filter_1['before_6'] = (df_filter_1.l_discount)
df_filter_1['before_7'] = (((df_filter_1.l_extendedprice) * (1 - (df_filter_1.l_discount))) * (1 + (df_filter_1.l_tax)))
df_group_1 = df_filter_1 \
    .groupby(['l_returnflag', 'l_linestatus']) \
    .agg(
        sum_before_1=("before_1", "sum"),
        mean_before_2=("before_2", "mean"),
        sum_before_3=("before_3", "sum"),
        min_before_4=("before_4", "min"),
        count_before_5=("before_5", "count"),
        min_before_6=("before_6", "min"),
        sum_before_7=("before_7", "sum"),
    )
df_group_1['sum_qty'] = ((df_group_1.sum_before_1 * df_group_1.mean_before_2) - 72)
df_group_1['sum_base_price'] = (df_group_1.sum_before_3 / (((df_group_1.count_before_5 - df_group_1.min_before_6) - df_group_1.min_before_4) * 0.5))
df_group_1['sum_charge'] = df_group_1.sum_before_7
df_group_1 = df_group_1[['sum_qty', 'sum_base_price', 'sum_charge']]

end_time = time.time()
print(df_group_1)

print("--- %s seconds ---" % (end_time - start_time))

                                     sum_qty    sum_base_price  \
l_returnflag l_linestatus                                        
A            F             943003.2507963751 76546.25946924335   
N            F             24759.73739653575  76568.9355216966   
             O            1915775.8957876577 76496.03121811729   
R            F             943099.2177945763 76501.70925219932   

                                  sum_charge  
l_returnflag l_linestatus                     
A            F             55909065222.82769  
N            F             1469649223.194375  
             O            113561024263.01378  
R            F             55889619119.83193  
--- 3.3769378662109375 seconds ---
